# Econox Quick Start

This notebook demonstrates the basic workflow of solving a Dynamic Programming problem using **Econox**.

Econox is designed to build and estimate economic models by combining three core components: **Environment (Model)**, **Physics (Logic)**, and **Computation (Solver)**.

> **概要:** このノートブックでは、Econox を使用して動的計画法（Dynamic Programming）の問題を解く基本的な流れを紹介します。Econox は「環境（Model）」、「物理法則（Logic）」、「計算機（Solver）」を組み合わせてモデルを構築します。

We will solve a simple **Dynamic Discrete Choice Model** with the following settings:

* **Model**: 10 states, 3 actions.
* **Utility**: Linear utility function ($U = \beta x + \epsilon$).
* **Solver**: Value Iteration.

> ここでは、10状態・3行動のシンプルな環境で、線形効用関数を持つ動的離散選択モデルを「価値反復法（Value Iteration）」を用いて解きます。

## 0. Install & Import

First, install the library and import the necessary modules.

> まずはライブラリをインストールし、モジュールをインポートします。

In [ ]:
%pip install econox

In [ ]:
import jax
import jax.numpy as jnp
import econox as ecx

## 1. Define the Environment (Model)

We define the "Environment" where the agents operate using `Model.from_data`. This container holds the state space size, action space size, and exogenous data (features, transition matrices).

> **1. 環境の定義**: 経済主体の置かれた環境を定義します。`Model.from_data` を使い、状態数、行動数、および外生的なデータ（特徴量や遷移確率）を格納したコンテナを作成します。

In [ ]:
# Settings
num_states = 10
num_actions = 3
key = jax.random.PRNGKey(0)

# Dummy Data Generation
# Feature 'x': Random values for each state-action pair (shape: S, A, 1)
x_data = jax.random.normal(key, (num_states, num_actions, 1))

# Transition Matrix: Uniform random transitions (shape: S*A, S)
# In a real model, this would be a sparse matrix or specific transition logic.
transitions = jnp.ones((num_states * num_actions, num_states)) / num_states

# Create Model using ecx.Model
model = ecx.Model.from_data(
    num_states=num_states,
    num_actions=num_actions,
    data={"x": x_data},
    transitions=transitions
)

print("Model created:", model)

## 2: Define the Agent (Solver)

Next, we define the **Agent** who makes decisions in this environment.

In **structural models**, the `Solver` typically represents the agent. It encapsulates:

1.  **Preferences (Utility):** What they value (e.g., Linear Utility).
2.  **Perception (Distribution):** How they perceive unobserved shocks (e.g., Gumbel/Logit).
3.  **Rationality (Algorithm):** How they calculate the optimal strategy (e.g., Value Iteration).

> **Note:** While we treat the Solver as an "Agent" in this example, the `Solver` interface itself is general. For reduced-form models or direct policy approximations, you can define a Solver that simply maps parameters to outcomes without explicit utility functions.

> **2: エージェント（Solver）の定義**
>
>次に、この環境で意思決定を行う「エージェント」を定義します。
>
>**構造モデルにおいて**、`Solver` は通常エージェントそのものを表します。Solverは以下を内包します。
>
>1.  **選好 (Utility):** 何を重視するか（例：線形効用）。
>2.  **認識 (Distribution):** 未観測の誤差をどう捉えるか（例：Gumbel分布によるLogitモデル）。
>3.  **合理性 (Algorithm):** どのように最適戦略を計算するか（例：価値反復法）。
>
>> **注釈:** 本例ではSolverを「エージェント」として扱いますが、Solverの仕組み自体は汎用的です。誘導形モデルや政策関数の直接近似など、効用関数を持たないモデルを定義することも可能です。

In [ ]:
# Define Utility Function
# U = param['beta'] * data['x']
utility = ecx.LinearUtility(param_keys=("beta",), feature_key="x")

# Define Error Distribution (Gumbel -> Logit Model)
dist = ecx.GumbelDistribution(scale=1.0)

# Define Solver
solver = ecx.ValueIterationSolver(
    utility=utility,
    dist=dist,
    discount_factor=0.95
)

## 3. Solve the Model

We provide the parameters and solve the model.
Calling `solver.solve()` performs the fixed-point iteration to find the equilibrium (Value Function and Choice Probabilities).

> **3. モデルを解く**: パラメータを与えてモデルを解きます。`solver.solve()` を呼び出すと、内部で不動点反復が行われ、価値関数と選択確率が計算されます。

In [ ]:
# Define Parameters
params = {"beta": jnp.array([1.0])}

# Solve
result = solver.solve(
    params=params,
    model=model
)

## 4. Check Results

The `SolverResult` object contains the solution (Value Function), the profile (Choice Probabilities), and convergence information.

> **4. 結果の確認**: `SolverResult` オブジェクトには、解（価値関数）、プロファイル（選択確率）、収束情報などが含まれています。

In [ ]:
print(f"Convergence Success: {result.success}")
print(f"Number of Iterations: {result.aux['num_steps']}")

print("\n--- Value Function (First 5 states) ---")
print(result.solution[:5])

print("\n--- Choice Probabilities (First 5 states) ---")
print(result.profile[:5])

## 5. Structural Estimation

Now, let's solve the inverse problem: **recovering the parameters from observed data.**

> **Note:** Forward-simulation capabilities (e.g., `model.simulate()`) are currently under active development. In this example, we generate synthetic data manually using JAX primitives.
> 
> (**注:** シミュレーション機能は現在開発中です。本例では JAX を使用して手動でデータを生成します。)

We will perform the following steps:
1.  **Generate Synthetic Data:** Simulate agent choices using the "True" model solved above ($\beta=1.0$).
2.  **Define Estimator:** Create an estimator with an initial guess ($\beta=0.0$).
3.  **Fit:** Run Maximum Likelihood Estimation to recover the true parameter.

> **5. 構造推定**: 最後に、逆問題（データからパラメータを推定する）を解いてみます。
> 1. **データの生成**: 先ほど解いた「真のモデル（$\beta=1.0$）」に基づいて、エージェントの選択データをシミュレーションします。
> 2. **Estimatorの定義**: 初期値を $\beta=0.0$ として推定器を定義します。
> 3. **推定**: 最尤法を実行し、真のパラメータが復元できるか確認します。

In [ ]:
# --- 1. Generate Synthetic Data (Simulation) ---
# Use the probabilities (result.profile) calculated in Step 4 with True Beta = 1.0

# Generate 1,000 observations to ensure statistical stability
num_obs = 1000
key, subkey_s, subkey_c = jax.random.split(key, 3)

# Randomly assign agents to states
state_indices = jax.random.randint(subkey_s, (num_obs,), 0, num_states)

# Sample choices based on the choice probabilities of those states
# result.profile: (num_states, num_actions)
probs = result.profile[state_indices]  # Shape: (num_obs, num_actions)
choice_indices = jax.random.categorical(subkey_c, jnp.log(probs))

observations = {
    "state_indices": state_indices,
    "choice_indices": choice_indices
}

print(f"Generated {num_obs} synthetic observations.")
print(f"True Parameter: beta = {params['beta']}")

In [ ]:
# --- 2. Define Estimator ---

# Define the parameter space with an initial guess (beta = 0.0)
# We want to see if it can move from 0.0 back to 1.0
param_space = ecx.ParameterSpace.create(initial_params={"beta": jnp.array([0.0])})

estimator = ecx.Estimator(
    model=model,
    param_space=param_space,
    method=ecx.MaximumLikelihood(),
    solver=solver   # Reuse the solver defined in Step 2
)

# --- 3. Run Estimation ---
est_result = estimator.fit(observations,sample_size=num_obs)

print(f"Estimation Success: {est_result.success}")
print(f"Estimated Parameter: {est_result.params['beta']}")
print(f"Error: {jnp.abs(est_result.params['beta'] - params['beta'])}")